<img src="./img/header.png">
# Globus SDK
https://github.com/globus/globus-sdk-python


# Globus SDK Docs
http://globus-sdk-python.readthedocs.io/en/1.5.0/


# Requirements
- You need to be in the tutorial users group for sharing: https://www.globus.org/app/groups/50b6a29c-63ac-11e4-8062-22000ab68755
- Installed Globus Python SDK

In [1]:
import ipywidgets as widgets
w = widgets.IntRangeSlider(
    value=[5, 7],
    min=0,
    max=10,
    step=1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)
from IPython.display import display
display(w)
widgets.IntSlider()
out = widgets.Output(layout={'border': '1px solid black'})
out

IntRangeSlider(value=(5, 7), continuous_update=False, description='Test:', max=10)

Output(layout=Layout(border='1px solid black'))

In [2]:
ep_petrel = "ddb59aef-6d04-11e5-ba46-22000b92c6ec"
ep_hli = "7c7f2228-719a-11e8-9363-0a6d4e044368"

# Configuration

The Globus Python SDK makes transfer functionality available via a TransferClient class.

We want to configure a TransferClient with an OAuth2 access token, to authenticate its connections with Globus.

In order to do this, we must first do a Native App Grant OAuth2 flow, in which we will
1. Login
2. Consent to allow Jupyter Notebook to access Globus Transfer on our behalf
3. Return to the notebook with an Authorization Code (the result of step 2)
4. Exchange the Authorization Code for an Access Token
5. Create a TransferClient object with that Access Token as its authorization method

In [3]:
import globus_sdk

# this is the ID of the Jupyter Demo App
CLIENT_ID = '3b1925c0-a87b-452b-a492-2c9921d3bd14'

# start Native App Grant, and print out the URL where users login as part of the flow (step 2 above)

# create a client object that tracks state as we do this flow
native_auth_client = globus_sdk.NativeAppAuthClient(CLIENT_ID)
# explicitly start the flow (some clients may support multiple flows)
native_auth_client.oauth2_start_flow()
# print URL
print("Login Here:\n\n{0}".format(native_auth_client.oauth2_get_authorize_url()))

print(("\n\nNote that this link can only be used once! "
       "If login or a later step in the flow fails, you must restart it."))

Login Here:

https://auth.globus.org/v2/oauth2/authorize?client_id=3b1925c0-a87b-452b-a492-2c9921d3bd14&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=openid+profile+email+urn%3Aglobus%3Aauth%3Ascope%3Atransfer.api.globus.org%3Aall&state=_default&response_type=code&code_challenge=xS8VW8piMxm1nBGMqEYRzYZnCQWAKse2dzRF_0vQf8w&code_challenge_method=S256&access_type=online


Note that this link can only be used once! If login or a later step in the flow fails, you must restart it.


### Come Back With Your Code

You'll now need to return with the copy-paste-able code that you got from login at the Authorize URL.
Insert it into the code block below, and it can be exchanged for a set of Access Tokens and Refresh Tokens (absent in this example).

In [4]:
# fill this line in with the code that you got
auth_code = "Ep1WJW0XuDWSSphsuPqoU13h0iyVXq"

# and exchange it for a response object containing your token(s)
token_response = native_auth_client.oauth2_exchange_code_for_tokens(auth_code)

# extract the Access Token for Globus Transfer, known as "transfer.api.globus.org"
transfer_access_token = token_response.by_resource_server['transfer.api.globus.org']['access_token']

# wrap the token in an object that implements the globus_sdk.GlobusAuthorizer interface
# in this case, an AccessTokenAuthorizer, which takes an access token and produces Bearer Auth headers
transfer_authorizer = globus_sdk.AccessTokenAuthorizer(transfer_access_token)

# create a TransferClient object which Authorizes its calls using that GlobusAuthorizer
tc = globus_sdk.TransferClient(authorizer=transfer_authorizer)

## Endpoint search

Globus has over 8000 registered endpoints. To find endpoints of interest you can access powerful search capabilities via the SDK. For example, to search for a given string across the descriptive fields of endpoints (names, description, keywords):

In [5]:
search_str = "Globus Tutorial Endpoint"
endpoints = tc.endpoint_search(search_str)
print("==== Displaying endpoint matches for search: '{}' ===".format(search_str))
for ep in endpoints:
    print("{} ({})".format(ep["display_name"] or ep["canonical_name"], ep["id"]))

==== Displaying endpoint matches for search: 'Globus Tutorial Endpoint' ===
Globus Tutorial Endpoint 1 (ddb59aef-6d04-11e5-ba46-22000b92c6ec)
Globus Tutorial Endpoint 2 (ddb59af0-6d04-11e5-ba46-22000b92c6ec)
Globus S3 Tutorial Endpoint (cf9bcaa5-6d04-11e5-ba46-22000b92c6ec)
Globus Tutorial HTTPS Endpoint Server (30ed71b0-fc1d-11e5-a700-22000bf2d559)
Brigitte's AWS Endpoint created for GlobusWorld2017 tutorial (a366d584-1f89-11e7-bc36-22000b9a448b)
Tutorial Endpoint 1 - Parsl Share (037f054a-15cf-11e8-b611-0ac6873fc732)
My Share on Globus Tutorial Endpoint 1 (28392b2c-3f1e-11e8-ba20-0ac6873fc732)
cbsnyder tutorial endpoint (5173cc8e-9b98-11e6-b0c6-22000b92c261)
NIH Tutorial Endpoint on AWS-JR (8a9188ac-e270-11e6-9d43-22000a1e3b52)
NIH Tutorial Endpoint - Xinyu (a6a7e6e2-e26d-11e6-9d43-22000a1e3b52)
Shared Tutorial Endpoint 2 (d0c79c14-3c5f-11e7-bcfe-22000b9a448b)
My Tutorial Shared Endpoint (3d60bef8-072a-11e6-a738-22000bf2d559)
My Tutorial Shared Endpoint2 (4ad8f450-0717-11e6-a738-2200

## Restricting search scope with filters

There are also a number of default filters to restrict the search for 'my-endpoints', 'my-gcp-endpoints',     'recently-used', 'in-use', 'shared-by-me','shared-with-me') 

In [6]:
search_str = None
endpoints = tc.endpoint_search(
        filter_fulltext=search_str, filter_scope="recently-used")
for ep in endpoints:
    print("{} ({})".format(ep["display_name"] or ep["canonical_name"], ep["id"]))

Amazon S3 Gateway - Vas (771717d0-b6f8-11e7-b106-22000a92523b)
Dan's tiny test point (978fc8ae-4d70-11e8-8fe1-0a6d4e044368)
ESnet Read-Only Test DTN at New York (d8eb3708-6d04-11e5-ba46-22000b92c6ec)
ESnet Read-Only Test DTN at Starlight (57218f41-3200-11e8-b907-0ac6873fc732)
ESnet Read-Only Test DTN at Sunnyvale (db57ddde-6d04-11e5-ba46-22000b92c6ec)
GCS v5.1 Video Share (c0d26edc-6f54-11e8-9327-0a6d4e044368)
Globus Tutorial Endpoint 1 (ddb59aef-6d04-11e5-ba46-22000b92c6ec)
Globus Tutorial Endpoint 2 (ddb59af0-6d04-11e5-ba46-22000b92c6ec)
Globus Tutorial HTTPS Endpoint Server (30ed71b0-fc1d-11e5-a700-22000bf2d559)
HLI Test (7c7f2228-719a-11e8-9363-0a6d4e044368)
UChicago RCC Midway (af7bda53-6d04-11e5-ba46-22000b92c6ec)
Vas Laptop (b5e4c04a-4d69-11e8-8fe0-0a6d4e044368)
XSEDE Stampede (e00a355f-6d04-11e5-ba46-22000b92c6ec)
XSEDE TACC stampede2 (ceea5ca0-89a9-11e7-a97f-22000a92523b)
globuspublish#trial_data (e4c16d8f-6d04-11e5-ba46-22000b92c6ec)
petrel#testbed (e56c36e4-1063-11e6-a747-22

## Endpoint details

You can also retrieve complete information about an endpoint, including name, owner, location, and server configurations. 

In [7]:
endpoint = tc.get_endpoint(tutorial_endpoint_1)
print("Display name:", endpoint["display_name"])
print("Owner:", endpoint["owner_string"])
print("ID:", endpoint["id"])

NameError: name 'tutorial_endpoint_1' is not defined

## Transfer

Creating a transfer is a two stage process. First you must create a description of the data you want to transfer (which also creates a unique submission_id), and then you can submit the request to Globus to transfer that data. 

If the submit_transfer fails, you can safely resubmit the same transfer_data again. The submission_id will ensure that this transfer request will be submitted once and only once.

In [ ]:
# help(tc.submit_transfer)
source_endpoint_id = tutorial_endpoint_1
source_path = "/share/godata/"

dest_endpoint_id = tutorial_endpoint_2
dest_path = "/~/"

label = "My tutorial transfer"

# TransferData() automatically gets a submission_id for once-and-only-once submission
tdata = globus_sdk.TransferData(tc, source_endpoint_id,
                                dest_endpoint_id,
                                label=label)

## Recursively transfer source path contents
tdata.add_item(source_path, dest_path, recursive=True)

## Alternatively, transfer a specific file
# tdata.add_item("/source/path/file.txt",
#                "/dest/path/file.txt"))

# Ensure endpoints are activated
tc.endpoint_autoactivate(source_endpoint_id)
tc.endpoint_autoactivate(dest_endpoint_id)

submit_result = tc.submit_transfer(tdata)
print("Task ID:", submit_result["task_id"])

## Get Task By ID

While the task is running, or after completion, you can get information that describes the transfer task. 

In [8]:
from datetime import datetime
hli_task_id = 'b1abfb20-7667-11e8-93d3-0a6d4e044368'
r = tc.get_task(hli_task_id)
print ("Label:", r["label"])
print ("Status:", r["status"])
print ("Transfer: {} -> {}".format(r["source_endpoint_display_name"],
                                   r["destination_endpoint_display_name"]))
    
if r.data["status"] == "SUCCEEDED":
    print("Bytes transferred:", r["bytes_transferred"])
    print("Files transferred:", r["files_transferred"])
    print("Transfer rate:", r["effective_bytes_per_second"], "bps")
    print("Start:", r['request_time'])
    print("End:", r['completion_time'])
    print("Duration:", datetime.strptime(r['completion_time'][:-6], '%Y-%m-%d %H:%M:%S') - datetime.strptime(r['request_time'][:-6], '%Y-%m-%d %H:%M:%S'))

Label: HLI_100x50GB 32-32 no verification
Status: SUCCEEDED
Transfer: petrel#testbed -> HLI Test
Bytes transferred: 5000000000000
Files transferred: 100
Transfer rate: 263443118 bps
Start: 2018-06-22 22:00:37+00:00
End: 2018-06-23 03:16:57+00:00
Duration: 5:16:20


In [10]:
#help(tc.task_event_list)
import json
r = tc.task_event_list(hli_task_id, num_results=1000)
mbps_total = 0
num_events = 0
for event in r:
    if event['code'] == "PROGRESS":
        print(event['time'],str(json.loads(event['details'])['mbps']) + " Mbps")
        mbps_total += json.loads(event['details'])['mbps']
        num_events += 1
print(mbps_total / num_events)


2018-06-23 03:16:57+00:00 274.01 Mbps
2018-06-23 03:15:58+00:00 323.98 Mbps
2018-06-23 03:14:58+00:00 835.79 Mbps
2018-06-23 03:13:58+00:00 1266.79 Mbps
2018-06-23 03:12:56+00:00 1377.66 Mbps
2018-06-23 03:11:55+00:00 1355.61 Mbps
2018-06-23 03:10:54+00:00 1341.82 Mbps
2018-06-23 03:09:54+00:00 1317.75 Mbps
2018-06-23 03:08:53+00:00 1245.35 Mbps
2018-06-23 03:07:53+00:00 1300.82 Mbps
2018-06-23 03:06:53+00:00 1376.58 Mbps
2018-06-23 03:05:53+00:00 1447.17 Mbps
2018-06-23 03:04:51+00:00 1472.5 Mbps
2018-06-23 03:03:50+00:00 1470.79 Mbps
2018-06-23 03:02:49+00:00 1397.87 Mbps
2018-06-23 03:01:49+00:00 1427.42 Mbps
2018-06-23 03:00:48+00:00 1281.03 Mbps
2018-06-23 02:59:48+00:00 1263.27 Mbps
2018-06-23 02:58:48+00:00 1097.23 Mbps
2018-06-23 02:57:46+00:00 1377.5 Mbps
2018-06-23 02:56:46+00:00 2537.12 Mbps
2018-06-23 02:55:46+00:00 3424.22 Mbps
2018-06-23 02:54:45+00:00 4200.28 Mbps
2018-06-23 02:53:44+00:00 4802.47 Mbps
2018-06-23 02:52:44+00:00 4838.73 Mbps
2018-06-23 02:51:44+00:00 4844

## Wait for task to complete

Transfer and delete tasks are asynchronous operations, and depending on their size may take a long time to complete. If you wish to wait for a task to complete, the TransferClient provides a task_wait helper method:

In [ ]:
# Wait for a task to finish for 10 minutes, polling every 15 seconds.
completed = tc.task_wait(submit_result["task_id"], timeout=600, polling_interval=15)
if completed:
    print("Task finished!")
else:
    print("Task still running after timeout reached.")
